In [32]:
import warnings
import cv2
warnings.simplefilter(action='ignore', category=FutureWarning)
from keras.models import load_model
from pytesseract import *
from PIL import Image
import cx_Oracle as cxi
import pandas as pd
import matplotlib.pyplot as plt
import folium
from plotnine import *
import math
import pyttsx3
from telegram.ext import Updater, MessageHandler, Filters, CommandHandler
from emoji import emojize
import os
import telegram

In [43]:
font = cv2.FONT_ITALIC
con = cxi.connect('Hyeonmo/1006@localhost:1521/xe')
cursor = con.cursor()  
def insertdata(img):
    image = Image.open(img)
    text = image_to_string(image, lang='kor')
    #print("=================존재하는 텍스트===============")
    name = text.split()[4]
    jumin = text.split()[5] + text.split()[6]
    address = ''
    for i in range(7, len(text.split())):
        address += text.split()[i]
    sql_insert = 'insert into cardinfo VALUES(:image, :name,:jumin ,:address)'
    cursor.execute(sql_insert, image='face.jpg', name=name, jumin=jumin, address=address)
    return text.split()[7:9]


In [44]:
def selectdata():
    cursor.execute('select * from cardinfo')
    cursor.fetchall()
    con.commit()
    cursor.close()
    con.close

In [45]:
def img_trim(img):  #숫자인식을 위해 이미지 trim
    x = 210
    y = 60
    w = 80
    h = 450
    img_trim = img[x:x+w, y:y+h]    #이미지 자르기
    cv2.imwrite('img_trim.jpg', img_trim)   #민증 숫자 부분 틀 저장
    return img_trim


In [46]:
def showimg(img):   #숫자 인식을 하기위해 모델을 가져와서 예측
    model = load_model('MNIST_CNN_model.h5')        #Model.py 를 통해 만든 모델
    model.summary()
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)    #회색으로 표시
    #plt.figure(figsize=(15, 12))
    img_blur = cv2.GaussianBlur(img_gray, (5, 5), 0)        #검은색 윤곽선 표시
    ret, img_th = cv2.threshold(img_blur, 100, 230, cv2.THRESH_BINARY_INV)  #원래 숫자를 저장
    image, contours, hierachy = cv2.findContours(img_th.copy(),     #숫자를 읽어오는 함수
                                                 cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    rects = [cv2.boundingRect(each) for each in contours]   #읽어온 숫자들을 저장
    tmp = [w * h for (x, y, w, h) in rects]     #각 숫자의 범위(0~9)
    tmp.sort()  #sort 안하면 저장 반대방향으로 출력
    rects = [(x, y, w, h) for (x, y, w, h) in rects if ((w * h > 70) and (w * h < 700))]       # 녹색 숫자 예상 테두리 부분(숫자인 부분만 예측하도록)
    img_result = []
    img_for_class = img.copy()
    margin_pixel = 11   #테두리 간의 사이 간격
    for rect in rects:
        #[y:y+h, x:x+w]
        target_num = img_for_class[rect[1] - margin_pixel: rect[1] + rect[3] + margin_pixel,        #녹색 테두리에 포함된 숫자 + margin 간격
                          rect[0]-margin_pixel: rect[0]+rect[2]+margin_pixel]
        test_num = cv2.resize(target_num, (28, 28))[:, :, 1]    #숫자 하나당 가로,세로28픽셀로 나누어 resize
        test_num = (test_num < 70) * test_num
        test_num = test_num.astype('float32') / 255
        test_num = test_num.reshape((1, 28, 28, 1))
        predicted_num = model.predict_classes(test_num)     #확대해서 나온 숫자를 model로 넣어서 학습
        print("=========예상 숫자==========")
        print(predicted_num)    #예상 숫자 출력
        img_result.append(
           img_for_class[rect[1] - margin_pixel: rect[1] + rect[3] + margin_pixel,
                          rect[0]-margin_pixel: rect[0]+rect[2]+margin_pixel])
        # Draw the rectangles
        cv2.rectangle(img, (rect[0], rect[1]),(rect[0] + rect[2], rect[1] + rect[3]), (0, 255, 0), 5)
        font = cv2.FONT_HERSHEY_SIMPLEX
        cv2.putText(img, str(predicted_num[0]), (rect[0], rect[1]), font, 1, (0, 0, 255), 5)    #위에 예상숫자 출력(파란색)
        #     plt.figure(figsize=(15, 12))
    plt.imshow(img)
    count = 0
    nrows = 6
    ncols = 6
    plt.figure(figsize=(12, 8))
    for n in img_result:
        count += 1
        plt.subplot(nrows, ncols, count)        #예측된 숫자들을 6*6으로 출력(윈도우로 보여줌)
        test_num = cv2.resize(img_result[0], (28, 28))[:, :, 1]
        test_num = (test_num < 70) * test_num
        test_num = test_num.astype('float32') / 255
        plt.imshow(cv2.resize(n, (28, 28)), cmap='Greys', interpolation='nearest')
        test_num = test_num.reshape((1, 28, 28, 1))
        plt.title(model.predict_classes(test_num))
    plt.tight_layout()
    plt.show()
    cv2.imwrite("result.jpg", img)      #예측 결과 빨간색으로 출력




In [47]:
def faceDetect():
    eye_detect = False
    face_detect = False
    face_cascade = cv2.CascadeClassifier("./haarcascade_frontalface_default.xml")   #얼굴인식용
    eye_cascade = cv2.CascadeClassifier("./haarcascade_eye.xml")    #눈 인식용용
    try:
        cam = cv2.VideoCapture(0)   #카메라 켜기
    except:
        print("camera loading error")
        return
    while True:
        ret, frame = cam.read()
        if not ret:
            break

        if eye_detect:
            info = "Eye Detention ON"
        else:
            info = "Eye Detection OFF"
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, 1.3, 5) #얼굴이 저장될 경우 얼굴 변수에 저장
        #카메라 영상 왼쪽 위에 세팅된 info 내용 출력
        cv2.putText(frame, info, (5, 15), font, 0.5, (255, 0, 255), 1)  #카메라 왼쪽위에 인식이 되면 ON/ 안되면 OFF
        cv2.putText(frame, "Enroll Card", (45, 195), font, 0.5, (255, 255, 255), 2) #흰색 틀 지정 위치를 지정(45,195)
        cv2.rectangle(frame, (50, 200), (400, 400), (255, 255, 255), 2)  # 사각형범위 지정
        cv2.imwrite("card.jpg", frame[200:350, 50:350]) #card.jpg가  enroll card 범위안에  들어올 때 캡처
        k = cv2.waitKey(27) #ESC 27일 떄 ESC를 누르면 종료
        for(x, y, w, h) in faces:
            cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 0, 0), 2)    #사각형
            cv2.imwrite("face.jpg", frame[y:y+h, x:x+w])  # 사각형 범위안에  들어올 때 캡처
            cv2.putText(frame, "Image Face", (x-5, y-5), font, 0.5, (0, 0, 0), 2)
            if eye_detect:      #눈이 인식될 경우 조건에 들어감
                roi_gray = gray[y:y+h, x:x+w]
                roi_color = frame[y:y+h, x:x+w]
                eyes = eye_cascade.detectMultiScale(roi_gray)
                for (ex, ey, ew, eh) in eyes:   #눈찾기
                    cv2.rectangle(roi_color, (ex, ey), (ex+ew, ey+eh), (0, 255, 0), 2)
                face_detect = True
                if face_detect:
                    break
        cv2.imshow("frame", frame)
        eye_detect = not eye_detect  #조건이 참이 되도록 조건
        if k == 27:
            break
    cam.release()   #카메라 끄는 명령문
    cv2.destroyAllWindows()


In [48]:
def text2speech(geo_df):
    geo_df=geo_df.sort_values(by=['거리'])
    tts=pyttsx3.init()
    tts.say(geo_df.iloc[0,31]+"에 있는" +geo_df.iloc[0,2]+"이 제일 가깝습니다",)
    tts.runAndWait()

In [49]:
def find(dong):     
    #si=dong[0:3]
    #gu=dong[3:6]
    #예시
    si='서울특별시'
    gu='종로구'
    apart= pd.read_excel('C:/Users/Hyeonmo/Desktop/Face/apart.xlsx', encoding='cp949')
    address_si = apart[apart['시']==si]
    address_gu = address_si[address_si['구']==gu]
    address_dong = address_gu[address_gu['동']=='혜화동']
    shop= pd.read_csv('C:/Users/Hyeonmo/Desktop/Face/seoul.csv', encoding='cp949')
    sanbon=shop[shop['행정동명'] == '혜화동']
    address_dong['위도'] = address_dong['위도'].astype(float)
    address_dong['경도'] = address_dong['경도'].astype(float)
    test=sanbon[sanbon['상권업종대분류명'].str.contains('')]
    test['위도'] = test['위도'].astype(float)
    test['경도'] = test['경도'].astype(float)
    test['거리']=0.00000000
    geo_df = test.copy()
    geo_address = address_dong.copy()
    map = folium.Map(location=[geo_address['위도'].mean(),geo_address['경도'].mean()],zoom_start=18)
    for n in geo_df.index:
        geo_df['거리'][n]=math.sqrt(math.pow(geo_address['위도'].mean()-geo_df['위도'][n],2)
                                                              + math.pow(geo_address['경도'].mean()-geo_df['경도'][n],2))
        popup_name=geo_df['상호명'][n] + '-' + geo_df['도로명주소'][n]
        if geo_df['상권업종대분류명'][n] == '음식':
            icon_color='red'
        elif geo_df['상권업종대분류명'][n] == '소매':
            icon_color='orange'
        elif geo_df['상권업종대분류명'][n] == '생활서비스':
            icon_color='yellow'
        elif geo_df['상권업종대분류명'][n] == '학문/교육':
            icon_color='green'
        elif geo_df['상권업종대분류명'][n] == '관광/여가/오락':
            icon_color='blue'
        elif geo_df['상권업종대분류명'][n] == '부동산':
            icon_color='purple'
        else:
            icon_color='gray'

        folium.CircleMarker(
                    location=[geo_df['위도'][n],geo_df['경도'][n]],
                    radius=3,
                    popup=popup_name,
                    color=icon_color,
                    fill=True,
                    fill_color=icon_color
                    ).add_to(map)    
    text2speech(geo_df)
    
    return map
            
                        
    

In [ ]:
#Main문
warnings.filterwarnings('ignore')
#faceDetect()
capture = "test.jpg"
img = cv2.imread(capture) #원본사진
trim_img = img_trim(img)    #이미지 자르기
showimg(trim_img)
result = insertdata(capture)
selectdata()
dong = ''
for temp in result:
    dong +=temp
find(dong)

In [46]:
#챗봇에서 동네를 알기 위한 메소드
def divgu(sen):
    sav=""
    if sen.find("구")>0:
        if len(sen)>sen.find("구")+1:
            sav= sen[sen.find("구")-2:sen.find("구")+1]
        else:
            sav=sen[sen.find("구")-2:sen.find("구")]+"구"
        return sav

'강남구'

In [ ]:
my_token='691858794:AAFP6ZfKxDu3hxiYF9HvrqRNhPf0c2pdt1o'
bot = telegram.Bot(token=my_token)
updates = bot.getUpdates()
chat_id=bot.getUpdates()[-1].message.chat.id   #최근에 보낸 메시지
#bot.sendMessage(chat_id=chat_id,text="저는 봇입니다")
print('start telegram chat bot')
dir_now = os.path.dirname(os.path.abspath('C:/Users/Hyeonmo/Desktop/Face/'))  # real path to dirname
#응답 function (MessageHandler에 의해 호출)
def get_Info(bot, update):
    dong= pd.read_excel('C:/Users/Hyeonmo/Desktop/Face/dong_report.xls', encoding='cp949')
    shop= pd.read_csv('C:/Users/Hyeonmo/Desktop/Face/seoul.csv', encoding='cp949')
    #bot.sendMessage(chat_id=chat_id,text="가게를 입력해주세요"+ emojize(':smiley:',use_aliases=True))
    gu=divgu(update.message.text)
    #위치정보
    for j in range(2,len(shop)):
        if shop.iloc[j,1] in update.message.text:   
            if gu in shop.iloc[j,31]:
                bot.sendMessage(chat_id=chat_id,text= shop.iloc[j,31]+ emojize(':smiley:',use_aliases=True)) 
            elif gu==None:
                bot.sendMessage(chat_id=chat_id,text= shop.iloc[j,31]+ emojize(':smiley:',use_aliases=True)) 
    
#명령 Function 
def help_command(bot,update):
    update.message.reply_text("가게를 입력해주세요")

# 사진 Function
def get_photo(bot, update) :
    file_path = os.path.join(dir_now, 'from_telegram.png')
    photo_id = update.message.photo[-1].file_id  # photo 번호가 높을수록 화질이 좋음
    photo_file = bot.getFile(photo_id)
    photo_file.download(file_path)
    update.message.reply_text('저장 완료')
    
#파일 Function(사용자)
def get_file(bot, update):
    file_id_short= update.message.document.file_id
    file_url= os.path.join(dir_now, update.message.document.file_name)
    bot.getFile(file_id_short).download(file_url)
    update.message.reply_text('저장 완료')


In [ ]:
updater=Updater(my_token)#봇의 업데이트 사항을 가져오는 클래스
address_handler=MessageHandler(Filters.text,get_address)#각종 메시지들을 다루기 위한 클래스
updater.dispatcher.add_handler(address_handler)

help_handler=CommandHandler('help',help_command)#각종 메시지들을 다루기 위한 클래스
updater.dispatcher.add_handler(help_handler)

photo_handler = MessageHandler(Filters.photo, get_photo)
updater.dispatcher.add_handler(photo_handler)

file_handler = MessageHandler(Filters.document, get_file)
updater.dispatcher.add_handler(file_handler)

updater.start_polling(timeout=3,clean=True)
updater.idle()  #updater가 종료되지 않고 계속 실행되어 있도록 하는 함수